In [1]:
import json
import csv


In [3]:
trainlines = open("../data/bagofwords/trainDataSeq2seq.tsv").readlines()
testlines = open("../data/bagofwords/testDataSeq2seq.tsv").readlines()
vallines = open("../data/bagofwords/devDataSeq2seq.tsv").readlines()

In [11]:
row=trainlines[0].strip().split("\t")

In [12]:
row

['If a snow storm is coming you should Buy supplies @@SEP@@ preparing for a storm requires predicting the occurrence of that storm',
 'Buy supplies is of preparing for a storm']

In [13]:
row[0]=row[0].replace("@@SEP@@","")
row

['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
 'Buy supplies is of preparing for a storm']

In [22]:
possible = set([word for word in row[0].split(" ")])
allowed = set([word for word in row[1].split(" ")])
notallowed = possible-allowed
print(len(allowed),len(notallowed))

8 11


In [20]:
dataset=[]
possible,allowed,notallowed
dataset = []
for word in allowed:
    dataset.append([row[0],word,1])
for word in notallowed:
    dataset.append([row[1],word,0])

In [21]:
dataset

[['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'of',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'preparing',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'supplies',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'is',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'storm',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'a',
  1],
 ['If a snow storm is coming you should Buy supplies  preparing for a storm requires predicting the occurrence of that storm',
  'Buy',
  1

In [123]:
import random
from nltk.corpus import stopwords
stpwords = set(stopwords.words('english'))
puncs = set([".","'","'s'",","])
removewords = stpwords.union(puncs)
def create_dataset_bagofwords(lines,fname):
    outfd = open("../data/bagofwords/"+fname,"w+")
    all_max = -1
    wordmap = {}
    posdataset = []
    negdataset = []
    for line in lines:
        row=line.strip().split("\t")
        row[0]=row[0].replace("@@SEP@@",".")
        possible = set([word for word in row[0].split(" ")])
        allowed = set([word for word in row[1].split(" ")])
        tnotallowed = possible-allowed
        tallowed = allowed  
        allowed = []
        notallowed = []
        for word in tallowed:
            if word.lower() not in removewords:
                allowed.append(word)
        for word in tnotallowed:
            if word.lower() not in removewords:
                notallowed.append(word)
        max_len = -1
        for word in allowed:
            if word =="":
                continue
            if word+"1" not in wordmap:
                wordmap[word+"1"]=1
            else:
                wordmap[word+"1"]+=1
            max_len = max(max_len,len(row[0].split(" "))+1)
            posdataset.append([row[0],word,1])
        for word in notallowed:
            if word =="":
                continue
            if word+"0" not in wordmap:
                wordmap[word+"0"]=1
            else:
                wordmap[word+"0"]+=1
            max_len = max(max_len,len(row[0].split(" "))+1)
            negdataset.append([row[0],word,0])
        all_max = max(max_len,all_max)

        
    len_pos = len(posdataset)
    len_neg = len(negdataset)
    final_dataset = []
    for dataset in [posdataset,negdataset]:
        for row in dataset:
            final_dataset.append(row)
    if len_pos>len_neg:
        diff = len_pos-len_neg
        for idx in range(0,diff):
            final_dataset.append(negdataset[random.randint(0,len_neg-1)])
    elif len_neg > len_pos:
        diff = len_neg-len_pos
        for idx in range(0,diff):
            final_dataset.append(posdataset[random.randint(0,len_pos-1)])

    for row in final_dataset:
        outfd.write("%s\t%s\t%d\n"%(row[0],row[1],row[2]))
        
    outfd.close()
    print(all_max)
    return wordmap

In [124]:
wordmap =  create_dataset_bagofwords(trainlines,"train.tsv")

84


In [125]:
w1= create_dataset_bagofwords(testlines,"test.tsv")
w2= create_dataset_bagofwords(vallines,"val.tsv")

79
71


In [126]:
import operator
sorted_x = sorted(wordmap.items(), key=operator.itemgetter(1))

In [127]:
sorted_x[-100:]

[('day0', 60),
 ('winter0', 60),
 ('get0', 60),
 ('available0', 61),
 ('surface0', 61),
 ('needs0', 61),
 ('survival0', 61),
 ('around0', 62),
 ('see0', 62),
 ('humans0', 63),
 ('like0', 63),
 ('temperature0', 63),
 ('sometimes0', 63),
 ('nutrients0', 63),
 ('become0', 64),
 ('moving0', 64),
 ('two0', 65),
 ('organisms0', 66),
 ('produce0', 66),
 ('air0', 66),
 ('process0', 66),
 ('place0', 66),
 ('resource0', 67),
 ('order0', 67),
 ('found0', 67),
 ('wants0', 69),
 ('large0', 69),
 ('area0', 70),
 ('sunlight0', 70),
 ('grow0', 70),
 ('plant1', 73),
 ('would1', 73),
 ('positive0', 73),
 ('plants1', 75),
 ('move0', 76),
 ('sun1', 76),
 ('changes0', 76),
 ('using0', 77),
 ('electrical0', 78),
 ('objects0', 78),
 ('electricity0', 78),
 ('another0', 79),
 ('find0', 79),
 ('might0', 79),
 ('new0', 80),
 ('heat1', 85),
 ('time0', 88),
 ('one0', 90),
 ('make0', 91),
 ('animals1', 91),
 ('change0', 94),
 ('occurs0', 94),
 ('usually0', 96),
 ('things0', 97),
 ('energy1', 97),
 ('formed0', 98),


In [133]:
set(["word"]).union(set(["w2"]))

{'w2', 'word'}

In [148]:
# Prepare scoring for bag-of-words Model
import csv
def create_score_dataset_for_bow(fname):
    lines = open("../data/with-genfact2/"+fname).readlines()
    ofd = open("../data/with-genfact2/"+fname+"-forscore.tsv","w+")
    csvout = csv.writer(ofd,delimiter='\t')
    for line in lines:
        row = line.strip().split("\t")
        base = row[0]
        hyp = row[1]
        fact = row[2]
        hyp_words = set(hyp.split(" "))
        fact_words = set(fact.split(" "))
        tall_words = hyp_words.union(fact_words)
        allowed=[]
        for word in tall_words:
            if word.lower() not in removewords:
                allowed.append(word)
        row.append("temp")
        for index,word in enumerate(allowed):
            idx = base+":"+str(index)
            row[0]=idx
            row[-1]= word
            csvout.writerow(row)
    ofd.close()

In [149]:
create_score_dataset_for_bow("test-trainedwithFact2.tsv")

In [150]:
create_score_dataset_for_bow("val-trainedwithFact2.tsv")

In [151]:
create_score_dataset_for_bow("top10-merged-testwithFact2.tsv")

In [152]:
create_score_dataset_for_bow("top10-merged-valwithFact2.tsv")

In [171]:
def create_post_scored_dataset_for_ir(fname):
    trainlines = open("../data/with-genfact2/"+fname+"-forscore.tsv").readlines()
    scorelines = open("../data/with-genfact2/"+fname+"-forscore.tsv-score.tsv").readlines()
    ofd = open("../data/with-genfact2/bowscored/"+fname,"w+")
    csvout = csv.writer(ofd,delimiter='\t')
    
    scoredict = {}
    for line in scorelines:
        line = line.strip().split("\t")
        scoredict[line[0]]=float(line[1])
    
    rows= {}
    for line in trainlines:
        row = line.strip().split("\t")
        base = row[0]
        hyp = row[1]
        fact = row[2]
        label = row[3]
        fscore = row[4]
        target = row[5]
        word = row[-1]
        score = scoredict[base]
        if score >= 0.4:
            idsplit = base.split(":")
            idx = idsplit[0]+":"+idsplit[1]+":"+idsplit[2]
            if idx not in rows:
                rows[idx]=[idx,hyp,fact,label,fscore,word]
            else:
                words = rows[idx][-1]
                words += " " + word
                rows[idx][-1] = words           
    
    for k,row in rows.items():
        csvout.writerow(row)

In [172]:
create_post_scored_dataset_for_ir("test-trainedwithFact2.tsv")
create_post_scored_dataset_for_ir("val-trainedwithFact2.tsv")
create_post_scored_dataset_for_ir("top10-merged-testwithFact2.tsv")
create_post_scored_dataset_for_ir("top10-merged-valwithFact2.tsv")

In [176]:
" ".join(list(set("predotor Predator lion lunch Lion".lower().split(" "))))

'predator predotor lion lunch'